In [2]:
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
%matplotlib inline

In [1]:
import main as main

In [14]:
main.run_all()

'Program has sucessfully scraped and saved the data.'

### Observing the Data that we have collected

In [15]:
all_titles = grab_titles()

In [72]:
meta_alabama = 0
alabama_titles = []
football_count = 0
football_titles = []
covid_count = 0
covid_titles = []
trump_count = 0
trump_titles = []
saban_count = 0
saban_titles = []
police_count = 0
police_titles = []
crime_count = 0
crime_titles = []
misc_titles = []
for i in set(all_titles):
    if 'Alabama' in i:
        meta_alabama +=1
        alabama_titles.append(i)
    elif 'football' in i:
        football_count += 1
        football_titles.append(i)
    elif 'Trump' in i:
        trump_count += 1
        trump_titles.append(i)
    elif 'Saban' in i:
        saban_count += 1
        saban_titles.append(i)
    elif 'police' in i:
        police_count += 1
        police_titles.append(i)
    else:
        misc_titles.append(i)

In [66]:
len(set(all_titles))

2085

In [73]:
meta_alabama

351

In [74]:
football_count

50

In [75]:
saban_count

28

In [76]:
trump_count

93

In [77]:
police_count

44

In [61]:
### There will be a few list that will make up the entire sports spectrum
### For example, there will be football words, baseball words, basketball words, etc.

# football_positions = ['QB','RB','FB','WR','OT','C','G','TE','DE','DT', 'LB', 'OLB', 'S', 'CB', 'FS']

# football_words = ['Saban', 'ESPN', 'Alabama football', 'Tide', 'Tide', 'football', 'NFL', 'Auburn football',
#                  'Nick Saban', 'cornerback', 'vs.', 'players', 'games', 'College Football']

# crime_words = ['murder', 'shooting', 'killed', 'dead', 'injured', 'homicide', 'gunfire', 'injured', 'beaten', 'death',
#               'arrested', 'abuse', 'charged']